<a href="https://colab.research.google.com/github/vohoaidanh/AIGCDetectBenchmark/blob/main/colab/intrinsic_inference.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Make sure to set the runtime to GPU: Runtime -> Change runtime type -> T4 GPU

You can upload your own images, then change the relevant code cells to load it and send it through the model.

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
# install the intrinsic decomposition repo from github
#!pip install https://github.com/compphoto/Intrinsic/archive/main.zip
!git clone https://github.com/compphoto/Intrinsic
%cd Intrinsic
!pip install .

Cloning into 'Intrinsic'...
remote: Enumerating objects: 131, done.
remote: Counting objects: 100% (18/18), done.
remote: Compressing objects: 100% (16/16), done.
remote: Total 131 (delta 11), reused 3 (delta 2), pack-reused 113
Receiving objects: 100% (131/131), 28.81 MiB | 32.56 MiB/s, done.
Resolving deltas: 100% (50/50), done.
/content/Intrinsic
Processing /content/Intrinsic
  Preparing metadata (setup.py) ... done
  Cloning https://github.com/CCareaga/MiDaS (to revision master) to /tmp/pip-install-ustdeqrv/altered-midas_f389dee33798432d97af843c9b180b5d
  Running command git clone --filter=blob:none --quiet https://github.com/CCareaga/MiDaS /tmp/pip-install-ustdeqrv/altered-midas_f389dee33798432d97af843c9b180b5d
  Resolved https://github.com/CCareaga/MiDaS to commit 043f3b8d9f6d2f7e740a8ac45e0745c0b5e2fd03
  Preparing metadata (setup.py) ... done
  Cloning https://github.com/CCareaga/chrislib (to revision main) to /tmp/pip-install-ustdeqrv/chrislib_0ac5716d0e714bc5a8c50d61763400e2


In [3]:
import torch

# import some helper functions from chrislib (will be installed by the intrinsic repo)
from chrislib.general import show, view, uninvert
from chrislib.data_util import load_image

# import model loading and running the pipeline
from intrinsic.pipeline import run_pipeline
from intrinsic.model_util import load_models

In [4]:
# download the pretrained weights and return the model (may take a bit to download weights)
DEVICE = 'cpu'
intrinsic_model = load_models('paper_weights', device=DEVICE)

Downloading: "https://github.com/compphoto/Intrinsic/releases/download/v1.0/final_weights.pt" to /root/.cache/torch/hub/checkpoints/final_weights.pt
100%|██████████| 485M/485M [00:03<00:00, 141MB/s]
Downloading: "https://github.com/facebookresearch/WSL-Images/zipball/main" to /root/.cache/torch/hub/main.zip
Downloading: "https://download.pytorch.org/models/ig_resnext101_32x8-c38310e5.pth" to /root/.cache/torch/hub/checkpoints/ig_resnext101_32x8-c38310e5.pth
100%|██████████| 340M/340M [00:10<00:00, 33.0MB/s]
/usr/local/lib/python3.10/dist-packages/torch/hub.py:294: UserWarning: You are about to download and run code from an untrusted repository. In a future release, this won't be allowed. To add the repository to your trusted list, change the command to {calling_fn}(..., trust_repo=False) and a command prompt will appear asking for an explicit confirmation of trust, or load(..., trust_repo=True), which will assume that the prompt is to be answered with 'yes'. You can also use load(..., 

In [ ]:
# load an example image from the github repo
torch.hub.download_url_to_file('https://raw.githubusercontent.com/compphoto/Intrinsic/main/figures/avocado.png', 'avo.png')

100%|██████████| 2.85M/2.85M [00:00<00:00, 30.7MB/s]


In [ ]:
# load the image to run through the pipeline
img = load_image('/content/Intrinsic/avo.png')

In [5]:
from PIL import Image
import numpy as np

In [ ]:
# run the image through the pipeline (use R0 resizing dicussed in the paper)
result = run_pipeline(
    intrinsic_model,
    img,
    resize_conf=0.0,
    maintain_size=True,
    linear=False,
    device=DEVICE
)

In [ ]:
# convert the inverse shading to regular shading for visualization
shd = uninvert(result['inv_shading'])
alb = result['albedo']

In [ ]:
shd3 = np.stack((shd, shd, shd), axis=-1)


In [ ]:
img_array = (shd * 255).astype(np.uint8)
img = Image.fromarray(img_array)
img.save("output_image.png")


In [ ]:
# show the result (gamma corrects the linear intrinsic components and scales to [0-1])
show([img, c, view(alb)], size=(20, 7))

## Intrinsic Process

In [ ]:
# Copy Data from gdrive to colab
!cp source -d dest


In [6]:
import os

def get_image_list(data_path):
  # Get all images in dataset (return a list of path of each image)
  items = os.listdir(data_path) # Each item contains 40 images
  image_list = []
  for item in items:
    f = os.path.join(data_path, item)
    imgs = os.listdir(f)
    imgs_1 = [os.path.join(f,i) for i in imgs]
    image_list.extend(imgs_1)
  return image_list


In [7]:
from PIL import Image
import os
import numpy as np
def save_image(image_, folder_path, filename):
    """
    Save an image to a specified folder.
    """
    image = (view(image_) * 255.0).astype(np.uint8)

    if image.shape[-1] == 1:
        image = np.squeeze(image, axis=2)

    if image.dtype != np.uint8:
        image = image.astype(np.uint8)

    image = Image.fromarray(image)

    # Create the folder if it doesn't exist
    if not os.path.exists(folder_path):
        os.makedirs(folder_path)

    # Construct the full file path
    file_path = os.path.join(folder_path, filename)
    print(file_path)
    # Save the image
    image.save(file_path)


In [8]:
!cp /content/drive/MyDrive/DATASETS/DiffusionForensics/imagenet/train/adm.tar.gz -d /content

In [9]:
!gunzip /content/adm.tar.gz

In [10]:
!tar -xvf /content/adm.tar

Streaming output truncated to the last 5000 lines.
adm/226/17.png
adm/226/3.png
adm/226/1.png
adm/226/25.png
adm/226/24.png
adm/226/34.png
adm/226/33.png
adm/226/23.png
adm/226/22.png
adm/226/18.png
adm/226/11.png
adm/226/39.png
adm/226/7.png
adm/226/27.png
adm/226/19.png
adm/226/0.png
adm/226/2.png
adm/226/29.png
adm/226/36.png
adm/226/20.png
adm/226/30.png
adm/226/8.png
adm/226/5.png
adm/226/14.png
adm/226/4.png
adm/226/9.png
adm/226/26.png
adm/226/31.png
adm/226/21.png
adm/226/32.png
adm/226/37.png
adm/226/38.png
adm/226/15.png
adm/226/16.png
adm/226/35.png
adm/226/6.png
adm/226/28.png
adm/226/12.png
adm/226/13.png
adm/956/
adm/956/10.png
adm/956/17.png
adm/956/3.png
adm/956/1.png
adm/956/25.png
adm/956/24.png
adm/956/34.png
adm/956/33.png
adm/956/23.png
adm/956/22.png
adm/956/18.png
adm/956/11.png
adm/956/39.png
adm/956/7.png
adm/956/27.png
adm/956/19.png
adm/956/0.png
adm/956/2.png
adm/956/29.png
adm/956/36.png
adm/956/20.png
adm/956/30.png
adm/956/8.png
adm/956/5.png
adm/956/14.p

In [11]:
data_path = '/content/Intrinsic/adm'
dest_root = '/content/drive/MyDrive/DATASETS/DiffusionForensics/intrinsic/imagenet/train/1_fake'
images = get_image_list(data_path)
from pathlib import Path


In [ ]:
images[:10]

['/content/Intrinsic/adm/897/35.png',
 '/content/Intrinsic/adm/897/16.png',
 '/content/Intrinsic/adm/897/12.png',
 '/content/Intrinsic/adm/897/26.png',
 '/content/Intrinsic/adm/897/17.png',
 '/content/Intrinsic/adm/897/10.png',
 '/content/Intrinsic/adm/897/27.png',
 '/content/Intrinsic/adm/897/3.png',
 '/content/Intrinsic/adm/897/4.png',
 '/content/Intrinsic/adm/897/13.png']

# Run process all images in the list

In [14]:
from tqdm import tqdm
print('Total: {0} images'.format(len(images)))
albedo = None
inv_shd = None
shading = None
DEVICE = 'cpu'
i = 1896 + 436
start = i
#images1 = images[:21] + images[1896:]
for image in tqdm(images[start:]):
    img = load_image(image)

    if img.shape[-1] == 3:
        result = run_pipeline(
          intrinsic_model,
          img,
          resize_conf=0.0,
          maintain_size=True,
          linear=False,
          device=DEVICE
        )

        i +=1
    else:
      print(image,':', img.shape)
      continue

    albedo = result['albedo']
    inv_shd = result['inv_shading']

    # compute shading from inverse shading
    shading = uninvert(inv_shd)
    # File name
    file_name = Path(image).stem

    save_image(albedo, dest_root,file_name + '_albedo_{0}'.format(i) + '.png')
    save_image(shading, dest_root,file_name + '_shading_{0}'.format(i) + '.png')
    save_image(img, dest_root,file_name + '_origin_{0}'.format(i) + '.png')




Total: 40000 images


  0%|          | 1/38104 [00:18<191:53:34, 18.13s/it]

/content/drive/MyDrive/DATASETS/DiffusionForensics/intrinsic/imagenet/train/1_fake/17_albedo_1897.png
/content/drive/MyDrive/DATASETS/DiffusionForensics/intrinsic/imagenet/train/1_fake/17_shading_1897.png
/content/drive/MyDrive/DATASETS/DiffusionForensics/intrinsic/imagenet/train/1_fake/17_origin_1897.png


  0%|          | 2/38104 [00:35<185:37:05, 17.54s/it]

/content/drive/MyDrive/DATASETS/DiffusionForensics/intrinsic/imagenet/train/1_fake/25_albedo_1898.png
/content/drive/MyDrive/DATASETS/DiffusionForensics/intrinsic/imagenet/train/1_fake/25_shading_1898.png
/content/drive/MyDrive/DATASETS/DiffusionForensics/intrinsic/imagenet/train/1_fake/25_origin_1898.png


...

/content/drive/MyDrive/DATASETS/DiffusionForensics/intrinsic/imagenet/train/1_fake/27_albedo_2331.png
/content/drive/MyDrive/DATASETS/DiffusionForensics/intrinsic/imagenet/train/1_fake/27_shading_2331.png
/content/drive/MyDrive/DATASETS/DiffusionForensics/intrinsic/imagenet/train/1_fake/27_origin_2331.png


  1%|          | 436/38104 [1:55:27<159:45:01, 15.27s/it]

/content/drive/MyDrive/DATASETS/DiffusionForensics/intrinsic/imagenet/train/1_fake/29_albedo_2332.png
/content/drive/MyDrive/DATASETS/DiffusionForensics/intrinsic/imagenet/train/1_fake/29_shading_2332.png
/content/drive/MyDrive/DATASETS/DiffusionForensics/intrinsic/imagenet/train/1_fake/29_origin_2332.png


  1%|          | 436/38104 [1:55:34<166:24:31, 15.90s/it]


KeyboardInterrupt: 